In [1]:
#!pip install openai
#!pip install pydub
#!pip install soundcard
#!pip install soundfile

#!pip uninstall whisper
#!pip install --user --force-reinstall openai-whisper==20230124

In [2]:
# Path to the file containing the API key
api_key_file = './api_key.txt'

# Function to read the API key from the file
def get_api_key(file_path):
    with open(file_path, 'r') as file:
        return file.read().strip()  # .strip() removes any leading/trailing whitespace

# Retrieve the API key from the file
OPENAI_API_KEY = get_api_key(api_key_file)
#display(openai.api_key)

In [3]:
#MODEL = "gpt-3.5-turbo"
MODEL = "gpt-4"

In [4]:
SAMPLE_RATE = 48000              # [Hz]. sampling rate
OUTPUT_FILE_NAME = "output.wav"    # file name.

In [5]:
# Recording duration
RECORD_MIN =  int(input("Enter time duration in minutes: "))*60 # [min]. duration recording audio.

Enter time duration in minutes: 3


In [6]:
import soundcard as sc
import soundfile as sf
from pydub import AudioSegment
import datetime
import os
import openai

print(openai.__version__)

C:\Users\Tim Wu\AppData\Roaming\Python\Python310\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


1.8.0


In [7]:
from openai import OpenAI

client = OpenAI(
  api_key=OPENAI_API_KEY,  # this is also the default, it can be omitted
)

In [8]:
with sc.get_microphone(id=str(sc.default_speaker().name), include_loopback=True).recorder(samplerate=SAMPLE_RATE) as mic:
    # record audio with loopback from default speaker.
    print("Recording.....n")
    data = mic.record(numframes=SAMPLE_RATE*RECORD_MIN)
    
    # change "data=data[:, 0]" to "data=data", if you would like to write audio as multiple-channels.
    sf.write(file=OUTPUT_FILE_NAME, data=data[:, 0], samplerate=SAMPLE_RATE)
    print("End Recording")

Recording.....n
End Recording


In [9]:
# Initialize an empty string to hold the transcript
transcript = ""

# load audio file
sound = AudioSegment.from_file(OUTPUT_FILE_NAME, format='wav')

# Set the segment length（unit：ms）
segment_length = 100000

# segment the audio file to multiple file
for i, chunk in enumerate(sound[::segment_length]):
    # Set the file name
    chunk.export(f'output_{i}.wav', format='wav')
    audio_file = open(f'output_{i}.wav', "rb")
    transcription = client.audio.transcriptions.create(
        model="whisper-1",  
        file=audio_file, 
        response_format="text")
    # Concatenate the resulting text with the existing transcript
    transcript += transcription + ' '
# Print the final transcript
# print(transcript)

In [10]:
transcript_ary = []

ret = ''
for script in transcript.split():
    ret = ret + ' ' + script
    if len(ret) > 1000:
        transcript_ary.append(ret)
        ret = ''
transcript_ary.append(ret)
#print(transcript_ary)

In [11]:
# Get the current date and time
now = datetime.datetime.now()
# Format the date and time as a string
timestamp = now.strftime("%Y-%m-%d_%H-%M-%S")
# Set the file name using the timestamp
filename = f"Raw_transcript_response_{timestamp}.txt"

# Print the name of the saved file
print(f"Data saved to {filename}.")

# Open the file in write mode and save the data
with open(filename, "w", encoding="utf-8") as f:
    for item in transcript_ary:
        f.write(item + "\n")

Data saved to Raw_transcript_response_2024-02-25_08-11-28.txt.


In [12]:
result_ary = []

for t in transcript_ary:
    completion = client.chat.completions.create(
      model=MODEL,
      messages=[
        {"role": "system", "content": "You are a helpful assistant and a professional writer. Your task is to summarize the text in bullets and sub-bullets format."},
        {"role": "user", "content": t}
      ]
    )

    result_ary.append(completion.choices[0].message)

In [13]:
final_result = ""
for res in result_ary:
    final_result += res.content + ' '
    print(res.content)

- Data about runners with muscle mass and glycogen consumption features are plotted in a dataset. 
  - Marathon runners are shown in red.
    - They are endurance athletes who need to be efficient and lean.
    - They tend to have a lower muscle mass and lower glycogen consumption than sprinters.
  - Sprinters are shown in blue.
    - They optimize for explosive power.
    - They usually have higher muscle mass and glycogen consumption than marathon runners.

- Glycogen consumption is plotted on the x-axis and muscle mass on the y-axis.
- A discriminative model, like logistic regression, is used to learn a decision boundary. 
  - The decision boundary separates data points into marathon runners and sprint runners.
  - It's represented by the black line on the plot.

- After fitting the data, the model can classify new data points as marathon or sprint runners based on their muscle mass and glycogen consumption.
- The text discusses the classification of athletes into sprint runners or 

In [14]:
# Get the current date and time
now = datetime.datetime.now()
# Format the date and time as a string
timestamp = now.strftime("%Y-%m-%d_%H-%M-%S")
# Set the file name using the timestamp
filename = f"Summary_response_{timestamp}.txt"

In [15]:
# Open the file in write mode and save the data
with open(filename, 'w', encoding='utf-8') as f:
    f.write(final_result)

# Print the name of the saved file
print(f"Data saved to {filename}.")

Data saved to Summary_response_2024-02-25_08-11-45.txt.
